# Sowing Success: How Machine Learning Helps Farmers Select the Best Crops

Measuring essential soil metrics such as nitrogen, phosphorous, potassium levels, and pH value is an important aspect of assessing soil condition. However, it can be an expensive and time-consuming process, which can cause farmers to prioritize which metrics to measure based on their budget constraints.

Farmers have various options when it comes to deciding which crop to plant each season. Their primary objective is to maximize the yield of their crops, taking into account different factors. One crucial factor that affects crop growth is the condition of the soil in the field, which can be assessed by measuring basic elements such as nitrogen and potassium levels. Each crop has an ideal soil condition that ensures optimal growth and maximum yield.

A farmer reached out to you as a machine learning expert for assistance in selecting the best crop for his field. They've provided you with a dataset called soil_measures.csv, which contains:

- "N": Nitrogen content ratio in the soil
- "P": Phosphorous content ratio in the soil
- "K": Potassium content ratio in the soil
- "pH" value of the soil
- "crop": categorical values that contain various crops (target variable).

Each row in this dataset represents various measures of the soil in a particular field. Based on these measurements, the crop specified in the "crop" column is the optimal choice for that field.

In this project, I will build multi-class classification models to predict the type of "crop" and identify the single most importance feature for predictive performance.

In [21]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import joblib

def load_data(file_path):
    """Load the dataset and perform basic checks."""
    df = pd.read_csv(file_path)
    print("First few rows of the dataset:")
    print(df.head())
    print("\nDataset Information:")
    print(df.info())
    print("\nMissing Values:")
    print(df.isnull().sum())
    print("\nBasic Statistics:")
    print(df.describe())
    print("\nDistribution of Target Variable (crop):")
    print(df['crop'].value_counts())
    return df

def handle_missing_values(df):
    """Handle missing values in the dataset."""
    return df.dropna()

def scale_features(df, features):
    """Scale the features using StandardScaler."""
    scaler = StandardScaler()
    return scaler.fit_transform(df[features]), scaler

def evaluate_feature_performance(df, features, target):
    """Evaluate the predictive performance of each feature individually."""
    feature_performance = {}
    for feature in features:
        print(f"\nEvaluating feature: {feature}")
        
        # Use the current feature as the only predictor
        X = df[[feature]]
        y = df[target]
        
        # Scale the feature
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Split the data into training and test sets
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
        
        # Train a Logistic Regression model
        model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=2000, random_state=42)
        model.fit(X_train, y_train)
        
        # Predict the target values for the test set
        y_pred = model.predict(X_test)
        
        # Evaluate the performance using accuracy
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Accuracy for feature {feature}: {accuracy}")
        
        # Store the performance in the dictionary
        feature_performance[feature] = accuracy
    return feature_performance

def find_best_feature(feature_performance):
    """Find the best predictive feature based on accuracy."""
    best_feature = max(feature_performance, key=feature_performance.get)
    best_score = feature_performance[best_feature]
    return {best_feature: best_score}

def train_and_evaluate_model(X, y):
    """Train and evaluate a multi-feature Logistic Regression model."""
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train the model
    model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=2000, random_state=42)
    model.fit(X_train, y_train)
    
    # Predict the target values for the test set
    y_pred = model.predict(X_test)
    
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    print("\nOverall Model Accuracy:", accuracy)
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    # Perform cross-validation
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    print("\nCross-Validation Accuracy:", scores.mean())
    
    return model

def save_model(model, file_name):
    """Save the trained model to a file."""
    joblib.dump(model, file_name)
    print(f"\nModel saved to {file_name}")

# Main script
if __name__ == "__main__":
    # File path to the dataset
    file_path = "soil_measures.csv"
    
    # Load the dataset
    df = load_data(file_path)
    
    # Handle missing values
    df = handle_missing_values(df)
    
    # Define features and target
    features = ['N', 'P', 'K', 'ph']
    target = 'crop'
    
    # Evaluate feature performance
    feature_performance = evaluate_feature_performance(df, features, target)
    print("\nFeature Performance:")
    for feature, accuracy in feature_performance.items():
        print(f"{feature}: {accuracy}")
    
    # Find the best predictive feature
    best_predictive_feature = find_best_feature(feature_performance)
    print("\nBest Predictive Feature:")
    print(best_predictive_feature)
    
    # Scale all features
    X_scaled, scaler = scale_features(df, features)
    y = df[target]
    
    # Train and evaluate the overall model
    model = train_and_evaluate_model(X_scaled, y)
    
    # Save the trained model
    save_model(model, "best_model.pkl")

First few rows of the dataset:
    N   P   K        ph  crop
0  90  42  43  6.502985  rice
1  85  58  41  7.038096  rice
2  60  55  44  7.840207  rice
3  74  35  40  6.980401  rice
4  78  42  42  7.628473  rice

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   N       2200 non-null   int64  
 1   P       2200 non-null   int64  
 2   K       2200 non-null   int64  
 3   ph      2200 non-null   float64
 4   crop    2200 non-null   object 
dtypes: float64(1), int64(3), object(1)
memory usage: 86.1+ KB
None

Missing Values:
N       0
P       0
K       0
ph      0
crop    0
dtype: int64

Basic Statistics:
                 N            P            K           ph
count  2200.000000  2200.000000  2200.000000  2200.000000
mean     50.551818    53.362727    48.149091     6.469480
std      36.917334    32.985883    50.647931     0.773938
min